In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.offline as po
import plotly.express as px
import os,sys
import json
from array import array

from collections import defaultdict
from scipy.signal import find_peaks
# Insert the path of modules folder 
# sys.path.insert(0, 'C:/Users/Jason/thesis_project')
sys.path.append("../")

from support import *
from features.feature_ts import genX
from experiment.algorithms.cluster_prep import *

C:\Users\Jason\AppData\Local\Programs\Python\Python39\lib\os.py
c:\Users\Jason\thesis_project\Jupyter_files\..


### Load energy consumption data:


In [2]:
year_range = [1997,2014]

df = genX(year_range)

### Identify daily peaks


In [5]:
Household = 3

X = df.reset_index()
X1 = X[X.ProfileID == Household]
X1.drop('ProfileID', axis = 1, inplace = True)
X2 = X1.set_index('date').T

new_df = []

for column in X2.columns:
    # print(X2[column])
    new_df.append(list(X2[column].values))

flat_list = [item for sublist in new_df for item in sublist]

X3 = pd.DataFrame(flat_list, columns = ['Vals'])

C:\Users\Jason\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [4]:
X2

date,1997-12-15,1997-12-16,1997-12-17,1997-12-18,1997-12-19,1997-12-20,1997-12-21,1997-12-22,1997-12-23,1997-12-24,...,1998-11-12,1998-11-13,1998-11-14,1998-11-15,1998-11-16,1998-11-17,1998-11-18,1998-11-19,1998-11-20,1998-11-21
0,4.396250,0.668250,0.812167,0.819167,0.748833,0.740000,0.802500,2.986500,4.473500,0.748500,...,0.002750,6.256500,5.341667,10.827000,3.866333,1.548583,5.737500,1.154583,1.219083,12.719667
1,0.673833,0.740333,0.751750,2.835167,0.996000,0.738083,0.935500,0.895417,2.168667,3.556167,...,0.002417,4.123917,6.179500,3.866667,4.145667,1.501250,1.787167,1.390250,1.247667,12.791083
2,0.801250,0.741583,1.382000,1.289583,1.349750,0.738917,0.733167,0.684667,1.744333,0.812000,...,0.002583,4.104250,3.971917,4.096000,3.936083,1.489250,1.486583,1.254000,1.390917,15.671500
3,0.808667,0.799250,6.623917,10.299250,4.972250,0.668750,6.147583,4.211083,4.980250,0.738583,...,0.002417,4.028500,4.043083,3.972667,3.998417,1.489333,4.700250,1.325333,1.148583,13.502333
4,0.813250,0.667583,7.606583,18.399250,15.080083,5.044250,13.615333,7.170000,10.442833,11.512833,...,0.002333,4.036000,4.094083,3.875667,6.143583,1.490083,1.483500,1.322500,1.227167,13.554250
5,0.802917,0.800750,9.273083,8.888667,5.327750,0.797583,1.813250,0.741083,9.736833,1.562833,...,0.002500,3.980833,4.027083,4.003667,3.914000,4.631833,1.547833,1.338667,1.304000,13.538917
6,0.811750,1.310417,0.680833,10.072917,5.113417,0.870917,0.657167,0.886250,1.216000,3.120250,...,15.788584,3.994333,3.961250,3.877917,4.009583,2.225833,1.693333,4.270000,5.912250,13.525000
7,1.048583,5.723500,0.818750,10.472667,0.833917,0.736333,0.758000,0.747167,2.587167,1.947333,...,12.329250,3.982167,3.872417,4.006667,4.746250,6.796500,2.636917,8.597750,7.909167,16.277000
8,3.035583,11.440583,0.757833,2.935583,0.841833,0.729417,0.825583,0.818000,1.609000,0.890583,...,16.058000,6.360500,4.608833,7.337667,3.938667,8.603250,5.175500,2.146083,1.466333,20.611167
9,0.747500,0.664833,1.358333,0.747917,3.705833,0.870750,0.799500,1.444250,0.933833,0.956500,...,13.486750,3.861583,7.148667,9.964917,3.069417,9.961750,2.751500,1.366500,1.626500,11.878500


In [4]:
fig = go.Figure(data=go.Scatter(
    y = X3.Vals,
    mode = 'lines'
))

fig.show()

In [2]:
from scipy.signal import find_peaks

indices = find_peaks(X3.Vals)[0]

fig = go.Figure()
fig.add_trace(go.Scatter(
    y=X3.Vals,
    mode='lines+markers',
    name='Original Plot'
))

fig.add_trace(go.Scatter(
    x=indices,
    y=[X3.Vals[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='cross'
    ),
    name='Detected Peaks'
))

fig.show()

NameError: name 'X3' is not defined

### Check how many loadprofile has only one reading

In [7]:
X = genX(year_range = [1994,2014], drop_0 = True)
zero_counter = 0

arrayX  = X.to_numpy().copy()
arrayY = X.to_numpy().copy()

arrayY.sort()

for i in range(0,len(arrayX)):
    temp= arrayY[i][len(arrayY[i])-2]

    if temp == 0:
        zero_counter = zero_counter + 1

zero_counter

dropping all zero rows


2400

### Check how many loadprofiles has two of the same amplitude peaks

In [8]:
peak1eqpeak2_counter = 0
peak2eqpeak3_counter = 0
peak1eqpeak2eqpeak3_counter = 0

arrayX  = X.to_numpy().copy()
arrayY = X.to_numpy().copy()

arrayY.sort()

for i in range(0,len(arrayX)):
    peak1= arrayY[i][len(arrayY[i])-1]
    peak2= arrayY[i][len(arrayY[i])-2]
    peak3= arrayY[i][len(arrayY[i])-3]

    if peak1 == peak2:
        peak1eqpeak2_counter = peak1eqpeak2_counter + 1
    if peak2 == peak3:
        peak2eqpeak3_counter = peak2eqpeak3_counter + 1
    if (peak1 == peak2):
        if peak1 == peak3:
            peak1eqpeak2eqpeak3_counter = peak1eqpeak2eqpeak3_counter + 1

print("There are {} profiles with peak1 == peak2".format(peak1eqpeak2_counter))
print("There are {} profiles with peak2 == peak3".format(peak2eqpeak3_counter))
print("There are {} profiles with peak1 == peak2 == peak 3".format(peak1eqpeak2eqpeak3_counter))



There are 38738 profiles with peak1 == peak2
There are 55211 profiles with peak2 == peak3
There are 23184 profiles with peak1 == peak2 == peak 3


### Check how many loadprofiles has only one non-zero peak

In [9]:
peak2null_counter = 0

# arrayX  = X.to_numpy().copy()
arrayY = X.to_numpy().copy()

arrayY.sort()

for i in range(0,len(arrayX)):

    peak2= arrayY[i][len(arrayY[i])-2]

    if peak2 == 0:
        peak2null_counter = peak2null_counter + 1


print("There are {} profiles with peak2 == 0".format(peak2null_counter))

There are 2400 profiles with peak2 == 0


## Identify loadprofiles peaks

### Solution 1 : Specify height of peaks and distance measures to try and find peaks

In [10]:
from collections import defaultdict

# Load data
year_range = [1994, 2014]
X = genX(year_range, drop_0= True)

df = X.copy()
arrayX  = X.to_numpy().copy()
arrayY = X.to_numpy().copy()
arrayY.sort()

dict_t = defaultdict(list)

X.reset_index(inplace = True)

for i in range(0,len(arrayX)):
    
    diff = arrayY[i][len(arrayY[i])-2] -arrayY[i][len(arrayY[i])-12]
    threshold = arrayY[i][len(arrayY[i])-2] - diff
    distance = 2

    indices = find_peaks(arrayX[i], distance=distance, height = threshold)[0]

    dict_t[i].append(indices)  

new = pd.DataFrame.from_dict(dict_t)

df_dict = new.T

X1 = pd.merge(X, df_dict, left_index = True, right_index = True)


dropping all zero rows


### Solution 2: Separate day into morning and afternoon and find the individual peaks

In [8]:
from collections import defaultdict

# Load data
year_range = [1994, 2014]
X = genX(year_range, drop_0= True)

df = X.copy()
arrayX  = X.to_numpy().copy()
arrayY = X.to_numpy().copy()

listYM  = list(map(lambda x : x[:12],arrayY))
listYA  = list(map(lambda x : x[12:],arrayY))

arrayYM = np.array(listYM)
arrayYA = np.array(listYA)

arrayYM.sort()
arrayYA.sort()

dict_M = defaultdict(list)
dict_A = defaultdict(list)

X.reset_index(inplace = True)

for i in range(0,len(arrayX)):
    
    diff_M = arrayYM[i][len(arrayYM[i])-1] - arrayYM[i][len(arrayYM[i])-2]
    diff_A = arrayYA[i][len(arrayYA[i])-1] - arrayYA[i][len(arrayYA[i])-2]

    threshold_M = arrayYM[i][len(arrayYM[i])-1] - diff_M/3
    threshold_A = arrayYA[i][len(arrayYA[i])-1] - diff_A/3
    
    # distance = 2

    indices_M = find_peaks(arrayX[i][0:12], height = threshold_M)[0]
    indices_A = find_peaks(arrayX[i][12:], height = threshold_A)[0]

    dict_M[i].append(indices_M) 
    dict_A[i].append(indices_A)

ToP_M_df = pd.DataFrame.from_dict(dict_M)
ToP_A_df = pd.DataFrame.from_dict(dict_A)

top_M_dict = ToP_M_df.T
top_A_dict = ToP_A_df.T + 12


X1 = X.merge(top_M_dict, left_index = True, right_index = True).merge(top_A_dict,left_index = True, right_index = True)
# X2 = X1.merge(top_A_dict,left_index = True, right_index = True)

dropping all zero rows


In [11]:
arrayY

array([[ 3.69166674,  0.80000002,  0.50833334, ...,  2.81666671,
         0.92499999,  2.02500007],
       [ 0.80833335,  0.50000001,  2.07500007, ...,  9.71666686,
         7.50000026,  0.84166666],
       [ 1.05833336,  1.62500005,  0.34166667, ..., 10.4333334 ,
         9.50833344,  4.40833342],
       ...,
       [ 0.38000003,  0.30666669,  0.28000002, ...,  2.99999996,
         0.90666667,  0.34000002],
       [ 0.29333335,  0.28666668,  0.28666668, ...,  0.6466667 ,
         0.63333336,  0.34000002],
       [ 0.35333336,  0.31333335,  0.25333335, ...,  0.6866667 ,
         0.64000004,  0.36000002]])

# Plot loadprofile and peaks

In [14]:
# indices = find_peaks(X1.loc[0][2:26], height = 0.6)[0]

# fig = go.Figure()
# fig.add_trace(go.Scatter(
#     y=X2.loc[0][2:26].values,
#     mode='lines+markers',
#     name='Original Plot'
# ))

# fig.add_trace(go.Scatter(
#     x=indices,
#     y=[X2.loc[0][2:26].values[j] for j in indices],
#     mode='markers',
#     marker=dict(
#         size=8,
#         color='red',
#         symbol='cross'
#     ),
#     name='Detected Peaks'
# ))

# fig.show()

## Remove household with less then 15 year electrified

In [15]:
# Load survey data

# xpath = os.path.join(survey_dir, 'delskv-' + str(year_range[0])+'-'+str(year_range[1])+'.csv') # fIX THE DIRECTROY ISSUA LATER
# ds = pd.read_csv(xpath)

ds = pd.read_csv(r'C:/Users/MEDIA_LAB/delskv-1994-2014-v1/delskv-1994-2014.csv')

ds['years_electrified'].fillna(0, inplace=True)

HouseholdIDs = list(ds[ds['years_electrified'] > 10]['ProfileID'])

# open output file for writing
with open('listfile.txt', 'w') as filehandle:
    json.dump(HouseholdIDs, filehandle)


def loadHouseholdIDs():
    # Load Household IDs
    with open('listfile.txt', 'r') as filehandle:
        HouseholdIDs = json.load(filehandle)# open output file for reading
        return HouseholdIDs



In [160]:
# Filter out all household with less than 10 years electrified
# X1 = X1[X1['ProfileID'].isin(loadHouseholdIDs())]

X1.rename(columns = {'0_x':'ToP_M', '0_y':'ToP_A'}, inplace=True)

# # Convert the ToP values to str
# newX['ToP'] = newX['ToP'].apply(lambda x: x.astype(str))

In [ ]:
# Check the number of peaks that the loadprofiles have
# list(set(newX['ToP'].apply(lambda x: len(x)).values))

### Split the ToP into morning and afternoon peaks
#### morning peaks = before 12
#### afternoon peaks = after 12

In [ ]:
# To be used with Solution 1 for finding morning and afternoon peaks
# fi = 12

# newX['ToP_M'] = newX['ToP'].apply(lambda x: x[x < fi])
# newX['ToP_A'] = newX['ToP'].apply(lambda x: x[x > fi])

### Combine all the morning/afternoon ToPs in one array for a household

In [9]:
X1.rename(columns = {'0_x':'ToP_M', '0_y':'ToP_A'}, inplace=True)

# Create series with all the morning peaks for each household
ToP_M = X1.groupby(['ProfileID'])['ToP_M'].apply(pd.Series.tolist)
ToP_A = X1.groupby(['ProfileID'])['ToP_A'].apply(pd.Series.tolist)

# Flatten the list of lists in the series
ToP_M = ToP_M.map(lambda x : [item for sublist in x for item in sublist])
ToP_A = ToP_A.map(lambda x : [item for sublist in x for item in sublist])

## Plot a distribution of a households time of peaks

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'figure.figsize':(7,5), 'figure.dpi':100})

# Plot Histogram on x
x = ToP_A[11]
plt.hist(x, bins=12)
plt.gca().set(title='Frequency Histogram', ylabel='Frequency');

### Identify hour of day on which most peaks occur

In [ ]:
# The below code returns a datafrmae with the household ID and the ToD in which most of the household morning and afternoon peaks occur

list_m = []

def tryconvert(value):
    try:
        return np.argmax(np.bincount(value))
    except (ValueError, TypeError):
        pass
    

#  x for item in sublist
list_m = ToP_M.map(lambda x: tryconvert(x))
list_a = ToP_A.map(lambda x: tryconvert(x))

ToP_A_df = list_a.to_frame()
ToP_M_df = list_m.to_frame()

ToP_df = ToP_M_df.merge(ToP_A_df, right_index=True, left_index=True)
ToP_df.head()

In [ ]:
index = 0

indices = X1.loc[index][-1].astype(int)

fig = go.Figure()
fig.add_trace(go.Scatter(
    y=X1.loc[index][2:26].values,
    mode='lines+markers',
    name='Original Plot'
))

fig.add_trace(go.Scatter(
    x=X1.loc[index][-1].astype(int),
    y=[X1.loc[index][2:26].values[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='cross'
    ),
    name='Detected Peaks'
))

fig.show()

### Identify all the loadprofiles morning and afternoon peaks

In [ ]:
X1

In [12]:
X1.rename(columns = {'0_x':'ToP_M', '0_y':'ToP_A'}, inplace=True)

# Create series with all the morning peaks for each household
ToP_M = X1.groupby(['ProfileID'])['ToP_M'].apply(pd.Series.tolist)
ToP_A = X1.groupby(['ProfileID'])['ToP_A'].apply(pd.Series.tolist)

# Flatten the list of lists in the series
ToP_M = ToP_M.map(lambda x : [item for sublist in x for item in sublist])
ToP_A = ToP_A.map(lambda x : [item for sublist in x for item in sublist])

In [6]:
series_ToP_M = ToP_M.apply(pd.Series).stack().reset_index(drop = True)
series_ToP_A = ToP_A.apply(pd.Series).stack().reset_index(drop = True)

In [13]:
X1['ToP'] = X1[['ToP_M', 'ToP_A']].values.tolist()
ToP = X1.groupby(['ProfileID','date'])['ToP'].apply(pd.Series.tolist)
temp = ToP.map(lambda x : [item for sublist in x for subsublist in sublist for item in subsublist])

ToPs = temp.apply(pd.Series).stack().reset_index(drop = True)

In [215]:
Temp1 = temp.to_frame()

In [ ]:
Temp1.reset_index(inplace = True)
Temp1

In [221]:
new_X1 = X1.merge(Temp1, right_index=True, left_index=True)

In [223]:
new_X1.drop(['ToP_x','ProfileID_y','date_y'], axis = 1, inplace=True)

In [ ]:
new_X1

In [ ]:
#Plot Histogram of "total_bill" with rugplot parameters
sns.histplot(ToPs, binwidth=1)
plt.title("Histogram of Time of Peaks") # for histogram title
plt.xlabel('Hour of day')

In [ ]:
from scipy import stats

#Plot Histogram of "total_bill" with label parameters
sns.distplot(series_ToP_M, fit = norm,kde = False,label="Time of peaks",bins = 10)
 
plt.title("Histogram of Time of Peaks") # for histogram title
plt.legend() # for label

In [ ]:
from scipy import stats

ax = sns.distplot(series_ToP_M, bins=10, kde=False, fit=stats.norm);
# plt.title('Distribution of Cointegrating Spread for Brent and Gasoil')

# Get the fitted parameters used by sns
(mu, sigma) = stats.norm.fit(series_ToP_M)
print("mu={0}, sigma={1}".format(mu, sigma))

# Legend and labels 
plt.legend(["normal dist. fit ($\mu=${0:.2g}, $\sigma=${1:.2f})".format(mu, sigma)])
plt.ylabel('Frequency')

# Cross-check this is indeed the case - should be overlaid over black curve
x_dummy = np.linspace(stats.norm.ppf(0.01), stats.norm.ppf(0.99), 100)
ax.plot(x_dummy, stats.norm.pdf(x_dummy, mu, sigma))
plt.legend(["normal dist. fit ($\mu=${0:.2g}, $\sigma=${1:.2f})".format(mu, sigma),
           "cross-check"])

## Create plot of amplitude vs ToP

In [14]:
# testing = X1.copy()
# testing['ToP'] = testing['ToP'].map(lambda x: [subsublist.astype(str) for sublist in x for subsublist in sublist])

In [16]:
X1['ToP'] = X1[['ToP_M', 'ToP_A']].values.tolist()
ToP = X1.groupby(['ProfileID','date'])['ToP'].apply(pd.Series.tolist)
temp = ToP.map(lambda x : [item for sublist in x for subsublist in sublist for item in subsublist])
Temp1 = temp.to_frame()
Temp1.reset_index(inplace = True)
new_X1 = X1.merge(Temp1, right_index=True, left_index=True)
new_X1.drop(['ToP_x','ProfileID_y','date_y'], axis = 1, inplace=True)
new_X1['ToP_y'] = new_X1['ToP_y'].map(lambda x: [sublist.astype(str) for sublist in x])

In [17]:
from plotly.subplots import make_subplots

temp = new_X1.sample(n =1000)

indices = temp['ToP_y']

fig = make_subplots(rows=2, cols=1, subplot_titles=("Amplitude vs Time of Peak", "Histogram of ToPs"))

# fig = go.Figure()
# fig.add_trace(go.Scatter(
#     x = [int(i) for i in indices[]],
#     y = temp.loc[0,indices[0]].values,
#     mode='markers',
#     name='Original Plot',
#     showlegend = False,   
# ))

def add_new_trace(index):
    fig.add_trace(go.Scatter(
    x=[int(i) for i in indices[index]],
    y=temp.loc[index,indices[index]].values,
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='circle'
    ),
    name='Detected Peaks',
    showlegend = False
    ))  

    # fig.add_trace(go.Scatter(
    #     x=[int(i) for i in indices[index]],
    #     y=temp.loc[index,indices[index]].values,
    #     mode='markers',
    #     marker=dict(
    #         size=5,
    #         color='red',
    #         symbol='circle'
    #     ),
    #     name='Detected Peaks'),
    #     row = 2,col = 1
    # )

for i in temp.index:
    add_new_trace(i)

fig.update_yaxes(title_text="Amplitude")

fig.update_xaxes(title_text = "Time of Peak")

fig.update_layout(title="Amplitude vs Time of Peak",
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 1
    )
)

fig.append_trace(go.Histogram(x = ToPs),row = 2, col =1)
fig.update_yaxes(title_text = "Count", row = 2, col = 1)
# fig.update(legend="false")

# my_layout = go.Layout(
#     autosize=False,
#     width=10,
#     height=1000,
#     showlegend = False)
fig.write_image("figure.png", engine = "kaleido")
# fig.show()

## Investigate ToP between small and large users

### Calculate the ToP and peaks

In [2]:
from collections import defaultdict


def determinePeaks(X):
    df = A.copy()
    arrayX  = A.to_numpy().copy()
    arrayY = A.to_numpy().copy()

    listYM  = list(map(lambda x : x[:12],arrayY))
    listYA  = list(map(lambda x : x[12:],arrayY))

    arrayYM = np.array(listYM)
    arrayYA = np.array(listYA)

    arrayYM.sort()
    arrayYA.sort()

    dict_M = defaultdict(list)
    dict_A = defaultdict(list)

    df.reset_index(inplace = True)

    for i in range(0,len(arrayX)):
        
        diff_M = arrayYM[i][len(arrayYM[i])-1] - arrayYM[i][len(arrayYM[i])-2]
        diff_A = arrayYA[i][len(arrayYA[i])-1] - arrayYA[i][len(arrayYA[i])-2]

        threshold_M = arrayYM[i][len(arrayYM[i])-1] - diff_M/3
        threshold_A = arrayYA[i][len(arrayYA[i])-1] - diff_A/3
        
        # distance = 2

        indices_M = find_peaks(arrayX[i][0:12], height = threshold_M)[0]
        indices_A = find_peaks(arrayX[i][12:], height = threshold_A)[0]

        dict_M[i].append(indices_M) 
        dict_A[i].append(indices_A)

    ToP_M_df = pd.DataFrame.from_dict(dict_M)
    ToP_A_df = pd.DataFrame.from_dict(dict_A)

    top_M_dict = ToP_M_df.T
    top_A_dict = ToP_A_df.T + 12


    X1 = df.merge(top_M_dict, left_index = True, right_index = True).merge(top_A_dict,left_index = True, right_index = True)
    # X2 = X1.merge(top_A_dict,left_index = True, right_index = True)

    X1.rename(columns = {'0_x':'ToP_M', '0_y':'ToP_A'}, inplace=True)

    X1['ToP'] = X1[['ToP_M', 'ToP_A']].values.tolist()
    ToP = X1.groupby(['ProfileID','date'])['ToP'].apply(pd.Series.tolist)
    temp = ToP.map(lambda x : [item for sublist in x for subsublist in sublist for item in subsublist])
    Temp1 = temp.to_frame()
    Temp1.reset_index(inplace = True)
    new_X1 = X1.merge(Temp1, right_index=True, left_index=True)
    new_X1.drop(['ToP_x'], axis = 1, inplace=True)
    new_X1['ToP_y'] = new_X1['ToP_y'].map(lambda x: [sublist.astype(str) for sublist in x])

    return new_X1

In [3]:
def calculateToPs(X1):
    
    X1['ToP'] = X1[['ToP_M', 'ToP_A']].values.tolist()
    ToP = X1.groupby(['ProfileID_y','date_y'])['ToP'].apply(pd.Series.tolist)
    temp = ToP.map(lambda x : [item for sublist in x for subsublist in sublist for item in subsublist])

    ToPs = temp.apply(pd.Series).stack().reset_index(drop = True)

    return ToPs

In [21]:
# def plotPeaks(X, bin)
from plotly.subplots import make_subplots

def plotToPs(A, bin):
    
    temp = determinePeaks(A)

    ToPs = calculateToPs(temp)

    indices = temp['ToP_y']

    fig = make_subplots(rows=2, cols=1, subplot_titles=("Amplitude vs Time of Peak", "Histogram of ToPs"))

    # fig = go.Figure()

    def add_new_trace(index):
        fig.append_trace(go.Scatter(
        x=[int(i) for i in indices[index]],
        y=temp.loc[index,indices[index]].values,
        mode='markers',
        marker=dict(
            size=8,
            color='red',
            symbol='circle'
        ),
        name='Detected Peaks',
        showlegend = False
        ),row =1, col =1)  

    for i in temp.index:
        add_new_trace(i)

    fig.update_yaxes(title_text="Amplitude")

    fig.update_xaxes(title_text = "Time of Peak")

    fig.update_layout(title="Profiles for bin " + str(bin),
        xaxis = dict(
            tickmode = 'linear',
            tick0 = 0,
            dtick = 1
        )
    )

    #Plot Histogram of "total_bill" with rugplot parameters
    fig.append_trace(go.Histogram(x=ToPs),row = 2, col =1)
    fig.update_yaxes(title_text="Count", row = 2, col = 1)
    # po.plot(fig, auto_open=False, filename=data_dir+'/plots/ToP_Amplitude_'+str(bin)+'.html')
    fig.write_image("figure" + str(bin)+".png", engine = "kaleido")

In [19]:
X = genX([1994,2014], drop_0 = True)

Xbin = xBins(X, 'amd')


dropping all zero rows


In [10]:
# for b, ids in Xbin.items():
#     print(b)
    
#     try:
#         A = X.loc[ids,:]
#     except:
#         A = ids

#     if len(A) < 1000:
#         print("1.Length of A is = {}".format(len(A)))
#         plotToPs(A,b)
#     else:	
#         temp = A.sample(n = 1000)
#         plotToPs(temp,b)
#         print("2.Length of A is = {}".format(len(A)))